In [ ]:
!pip install Augmentor
import Augmentor
!pip install split-folders
import splitfolders
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG19
from keras.optimizers import Adam
from PIL import Image
import sys

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def augment_images(folder, max_count=0, in_max_count=0):
    for animal in folder:
        num_of_augmented = 0
        augmentator = Augmentor.Pipeline(animal, output_directory="")
        if (max_count==0):
            if (in_max_count!=0):
                num_of_augmented = in_max_count * len(augmentator.augmentor_images)
            else: num_of_augmented = 0
        else: num_of_augmented = max_count
        p_flip = 0.68
        augmentator.flip_left_right(probability=p_flip)
        p_skew = 0.63
        augmentator.skew_left_right(probability=p_skew, magnitude=0.3)
        p_rotate = 0.7
        augmentator.rotate(probability=p_rotate, max_left_rotation=15, max_right_rotation=15)
        p_zoom = 0.65
        augmentator.zoom(probability=p_zoom, min_factor=1.1, max_factor=1.25)
        p_erase = 0.45
        augmentator.random_erasing(probability=p_erase,rectangle_area=0.23)
        p_bright = 0.55
        augmentator.random_brightness(probability=p_bright, min_factor=1.15, max_factor=1.3)
        p_contrast = 0.55
        augmentator.random_contrast(probability=p_contrast, min_factor=1.1, max_factor=1.25)      
        augmentator.sample(num_of_augmented)

In [ ]:
augment_images(['/content/drive/MyDrive/coffee/coffeedataset40'], max_count=1000)

Initialised with 80 image(s) found.
Output directory set to /content/drive/MyDrive/coffee/coffeedataset40/.

Processing <PIL.Image.Image image mode=RGB size=256x256 at 0x7F7FF98169A0>: 100%|██████████| 1000/1000 [00:29<00:00, 33.35 Samples/s]


In [ ]:
output = "sortedData"
splitfolders.ratio('/content/drive/MyDrive/coffee/coffeedataset40', output=output, seed=42, ratio=(0.7, 0.15, 0.15))

Copying files: 1080 files [00:03, 297.67 files/s]


In [ ]:
tf.compat.v1.enable_eager_execution()

In [ ]:
train_dir = "sortedData/train"
test_dir = "sortedData/test"
val_dir = "sortedData/val"
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 256
nb_train_samples = 756
nb_validation_samples = 162
nb_test_samples = 162

In [ ]:
vgg19_net = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
vgg19_net.trainable = False
vgg19_net.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model = Sequential()
model.add(vgg19_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               6422784   
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
 activation_3 (Activation)   (None, 1)                

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-5), 
              metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
    )

Found 767 images belonging to 2 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
    )

Found 165 images belonging to 2 classes.


In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
    )

Found 168 images belonging to 2 classes.


In [ ]:
sys.modules['Image'] = Image
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
2/2 [==============================] - 411s 201s/step - loss: 0.7387 - accuracy: 0.5527
Epoch 2/5
2/2 [==============================] - 415s 214s/step - loss: 0.6913 - accuracy: 0.5323
Epoch 3/5
2/2 [==============================] - 402s 200s/step - loss: 0.6404 - accuracy: 0.6301
Epoch 4/5
2/2 [==============================] - 403s 200s/step - loss: 0.6193 - accuracy: 0.6426
Epoch 5/5
2/2 [==============================] - 409s 204s/step - loss: 0.5358 - accuracy: 0.7578


In [ ]:
vgg19_net.trainable = True
trainable = False
for layer in vgg19_net.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    layer.trainable = trainable
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               6422784   
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
 activation_3 (Activation)   (None, 1)                

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=2,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
2/2 [==============================] - 513s 254s/step - loss: 0.4946 - accuracy: 0.7793
Epoch 2/2
2/2 [==============================] - 503s 254s/step - loss: 0.4177 - accuracy: 0.8337


In [ ]:
scores = model.evaluate_generator(test_generator)
scores

<ipython-input-50-99161fc214a4>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator)


[0.3441554307937622, 0.9285714030265808]

In [ ]:
print("test data accuracy: %.2f%%" % (scores[1]*100))

test data accuracy: 92.86%
